In [22]:
import pandas as pd
import duckdb
import requests
import tempfile
import random
from IPython.display import clear_output
import json

import sys

colab = False
if colab:
    !pip install datasets
    clear_output(wait=True)
    import sys
    sys.path.append("./src")
    import env_options
    import lmsys_dataset_handler as lmsys
    from google.colab import userdata
    colab_secrets = {'HF_TOKEN':userdata.get('HF_TOKEN'),
                    'HF_TOKEN_WRITE':userdata.get('HF_TOKEN_WRITE')}
    hf_token, hf_token_write = env_options.check_env(colab=True, use_dotenv=False, colab_secrets=colab_secrets)

else:
    sys.path.append("./src")
    import env_options
    import lmsys_dataset_handler as lmsys
    dotenv_path = "../../apis/.env"
    hf_token, hf_token_write = env_options.check_env(colab=False, use_dotenv=True, dotenv_path=dotenv_path)

Python version: 3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)]
PyTorch version: 2.2.2
Transformers version: 4.44.2
CUDA device: NVIDIA GeForce RTX 4060 Laptop GPU
CUDA Version: 12.1
FlashAttention available: True
Retrieved HuggingFace token(s) from .env file
Using HuggingFace token: hf_M*****************************IASJ
Using HuggingFace write token: hf_u*****************************Xipx


### Checking Parquet files of a dataset

### Checking Parquet file URLs

In [5]:
headers = {"Authorization": f"Bearer {hf_token}"}
url = 'https://huggingface.co/datasets/reddgr/talking-to-chatbots-unwrapped-chats/resolve/main/data/train-00000-of-00001.parquet'
dataset_name = "reddgr/talking-to-chatbots-unwrapped-chats"
API_URL = f"https://datasets-server.huggingface.co/parquet?dataset={dataset_name}"
response = requests.get(API_URL, headers=headers)
print(response.json())

{'parquet_files': [{'dataset': 'reddgr/talking-to-chatbots-unwrapped-chats', 'config': 'default', 'split': 'train', 'url': 'https://huggingface.co/datasets/reddgr/talking-to-chatbots-unwrapped-chats/resolve/refs%2Fconvert%2Fparquet/default/train/0000.parquet', 'filename': '0000.parquet', 'size': 7717425}], 'pending': [], 'failed': [], 'partial': False}


In [ ]:


headers = {"Authorization": f"Bearer {hf_token}"}
dataset_name = "lmsys/lmsys-chat-1m"
API_URL = f"https://datasets-server.huggingface.co/parquet?dataset={dataset_name}"
response = requests.get(API_URL, headers=headers)
print(json.dumps(response.json(), indent=2))

{
  "parquet_files": [
    {
      "dataset": "lmsys/lmsys-chat-1m",
      "config": "default",
      "split": "train",
      "url": "https://huggingface.co/datasets/lmsys/lmsys-chat-1m/resolve/refs%2Fconvert%2Fparquet/default/train/0000.parquet",
      "filename": "0000.parquet",
      "size": 249303811
    },
    {
      "dataset": "lmsys/lmsys-chat-1m",
      "config": "default",
      "split": "train",
      "url": "https://huggingface.co/datasets/lmsys/lmsys-chat-1m/resolve/refs%2Fconvert%2Fparquet/default/train/0001.parquet",
      "filename": "0001.parquet",
      "size": 247222671
    },
    {
      "dataset": "lmsys/lmsys-chat-1m",
      "config": "default",
      "split": "train",
      "url": "https://huggingface.co/datasets/lmsys/lmsys-chat-1m/resolve/refs%2Fconvert%2Fparquet/default/train/0002.parquet",
      "filename": "0002.parquet",
      "size": 249923890
    },
    {
      "dataset": "lmsys/lmsys-chat-1m",
      "config": "default",
      "split": "train",
      "url

In [16]:
url = "https://huggingface.co/datasets/lmsys/lmsys-chat-1m/resolve/refs%2Fconvert%2Fparquet/default/train/0000.parquet"
head_response = requests.head(url, allow_redirects=True, headers=headers)
file_size = int(head_response.headers['Content-Length'])
print(f"{file_size} bytes")

249303811 bytes


In [12]:
url = f'https://huggingface.co/datasets/lmsys/lmsys-chat-1m/resolve/main/data/train-00000-of-00006-4feeb3f83346a0e9.parquet'
head_response = requests.head(url, allow_redirects=True, headers=headers)
file_size = int(head_response.headers['Content-Length'])
print(f"train-00000-of-00006-4feeb3f83346a0e9.parquet: {file_size} bytes")

train-00000-of-00006-4feeb3f83346a0e9.parquet: 249303811 bytes


### Querying Parquet files

In [21]:
url = 'https://huggingface.co/datasets/reddgr/talking-to-chatbots-unwrapped-chats/resolve/refs%2Fconvert%2Fparquet/default/train/0000.parquet'
query = """
        SELECT * FROM read_parquet('{url}') USING SAMPLE 1
        """
query = query.format(url=url)
query_result = duckdb.query(query).df()
display(query_result)

,conversation_id,turn,prompt,response,category,language,pred_label_rq,prob_rq,pred_label_tl,prob_tl,model,message_tag,date,turns,source,chatbot_id,chatbot_name,attachments,conversation_tag
0,34584705-62c0-45c6-82ba-33b1d147a05e,2,find more,Certainly! Here are a few more emoji combinati...,General Knowledge,en,request,0.975803,test,0.581291,gpt-4-gizmo,,2023-09-27,2,chatgpt,,,[],


In [17]:
url = 'https://huggingface.co/datasets/reddgr/talking-to-chatbots-unwrapped-chats/resolve/refs%2Fconvert%2Fparquet/default/train/0000.parquet'
query = """
        SELECT * FROM read_parquet('{url}') LIMIT 2
        """
query = query.format(url=url)
query_result = duckdb.query(query).df()
display(query_result)

,conversation_id,turn,prompt,response,category,language,pred_label_rq,prob_rq,pred_label_tl,prob_tl,model,message_tag,date,turns,source,chatbot_id,chatbot_name,attachments,conversation_tag
0,99acfac8-2c78-4ab6-964d-5d63449fd539,1,Help me rephrase this tagline by giving me a f...,"Sure, I can help you with that. Here are some ...",Culture and Entertainment,en,request,0.976421,learn,0.956283,openai-gpt,,2023-06-01,12,bing-skype,,,[],
1,99acfac8-2c78-4ab6-964d-5d63449fd539,2,Give me a creative sentence for a closed ended...,Here are some creative sentences for a closed ...,Language and writing,en,request,0.985784,learn,0.953707,openai-gpt,,2023-06-01,12,bing-skype,,,[],


In [8]:
url = 'https://huggingface.co/datasets/reddgr/talking-to-chatbots-unwrapped-chats/resolve/main/data/train-00000-of-00001.parquet'
query = """
        SELECT * FROM read_parquet('{url}') LIMIT 2
        """
query = query.format(url=url)
query_result = duckdb.query(query).df()
display(query_result)

,conversation_id,turn,prompt,response,category,language,pred_label_rq,prob_rq,pred_label_tl,prob_tl,model,message_tag,date,turns,source,chatbot_id,chatbot_name,attachments,conversation_tag
0,99acfac8-2c78-4ab6-964d-5d63449fd539,1,Help me rephrase this tagline by giving me a f...,"Sure, I can help you with that. Here are some ...",Culture and Entertainment,en,request,0.976421,learn,0.956283,openai-gpt,,2023-06-01,12,bing-skype,,,[],
1,99acfac8-2c78-4ab6-964d-5d63449fd539,2,Give me a creative sentence for a closed ended...,Here are some creative sentences for a closed ...,Language and writing,en,request,0.985784,learn,0.953707,openai-gpt,,2023-06-01,12,bing-skype,,,[],


### Querying LMSYS files

In [16]:
data_files = [
    "train-00000-of-00006-4feeb3f83346a0e9.parquet",
    "train-00001-of-00006-4030672591c2f478.parquet",
    "train-00002-of-00006-1779b7cec9462180.parquet",
    "train-00003-of-00006-2fa862bfed56af1f.parquet",
    "train-00004-of-00006-18f4bdd50c103e71.parquet",
    "train-00005-of-00006-fe1acc5d10a9f0e2.parquet"
]


for file in data_files:
    url = f'https://huggingface.co/datasets/lmsys/lmsys-chat-1m/resolve/main/data/{file}'
    head_response = requests.head(url, allow_redirects=True, headers=headers)
    file_size = int(head_response.headers['Content-Length'])
    print(f"{file}: {file_size} bytes")

train-00000-of-00006-4feeb3f83346a0e9.parquet: 249303811 bytes
train-00001-of-00006-4030672591c2f478.parquet: 247222671 bytes
train-00002-of-00006-1779b7cec9462180.parquet: 249923890 bytes
train-00003-of-00006-2fa862bfed56af1f.parquet: 247173225 bytes
train-00004-of-00006-18f4bdd50c103e71.parquet: 246443273 bytes
train-00005-of-00006-fe1acc5d10a9f0e2.parquet: 248783380 bytes


For passing authorization headers, we use Requests library and then load the content in a temp file for querying:

In [17]:
url = f'https://huggingface.co/datasets/lmsys/lmsys-chat-1m/resolve/main/data/{data_files[1]}'
print(url)
# Download file with auth headers using requests
r = requests.get(url, headers=headers)
# Write the downloaded content into a temporary file
with tempfile.NamedTemporaryFile(suffix=".parquet", delete=False) as tmp:
	tmp.write(r.content)
	tmp_path = tmp.name
# Query using DuckDB from the temporary file
query_result = duckdb.query(f"SELECT * FROM read_parquet('{tmp_path}') LIMIT 2").df()
display(query_result)

https://huggingface.co/datasets/lmsys/lmsys-chat-1m/resolve/main/data/train-00001-of-00006-4030672591c2f478.parquet


,conversation_id,model,conversation,turn,language,openai_moderation,redacted
0,23828586342d4baca3cfd731650b78bc,vicuna-13b,"[{'content': 'You can say nsfw? ', 'role': 'us...",1,English,"[{'categories': {'harassment': False, 'harassm...",False
1,25cfe27cf42a487aa0c2b574ffbea283,vicuna-13b,"[{'content': 'Hello, we met with my girlfriend...",3,English,"[{'categories': {'harassment': False, 'harassm...",True


In [18]:
random.shuffle(data_files)
url = f'https://huggingface.co/datasets/lmsys/lmsys-chat-1m/resolve/main/data/{data_files[0]}'
print(url)
r = requests.get(url, headers=headers)
with tempfile.NamedTemporaryFile(suffix=".parquet", delete=False) as tmp:
	tmp.write(r.content)
	tmp_path = tmp.name

query = """
    SELECT * FROM read_parquet('{tmp_path}') 
    WHERE contains(lower(cast(conversation as VARCHAR)), 'mounting an');
    """
query = query.format(tmp_path=tmp_path)
query_result = duckdb.query(query).df()
display(query_result)

https://huggingface.co/datasets/lmsys/lmsys-chat-1m/resolve/main/data/train-00002-of-00006-1779b7cec9462180.parquet


,conversation_id,model,conversation,turn,language,openai_moderation,redacted
0,41118783e4d348cd9f2fc1782e61bc00,claude-1,[{'content': 'Error mounting system-managed de...,2,English,"[{'categories': {'harassment': False, 'harassm...",False
1,13f156f4620a49469fd842e0dfcffab7,claude-1,[{'content': 'How can I run multiple instances...,8,English,"[{'categories': {'harassment': False, 'harassm...",True
2,e4c677346bf24ed8872430dcb86b4652,vicuna-33b,[{'content': 'Write a explicit story about a m...,2,English,"[{'categories': {'harassment': False, 'harassm...",True
3,e4670455dc894720b3a5ee66573d922d,wizardlm-13b,[{'content': 'ten steps (bulleted format) desc...,1,English,"[{'categories': {'harassment': False, 'harassm...",False
4,d3af302653544d878add18c658cb6131,chatglm-6b,[{'content': 'But luks is supported by most di...,5,English,"[{'categories': {'harassment': False, 'harassm...",False
5,4d9e81e1fb224b78bbed8f94761f74c8,wizardlm-13b,[{'content': 'write a short story of an except...,2,English,"[{'categories': {'harassment': False, 'harassm...",True
6,fd7b307bb46245d2bd654d63833b2a99,vicuna-33b,[{'content': 'Write a explicit story about a m...,2,English,"[{'categories': {'harassment': False, 'harassm...",True
7,bb99240a239f4c8fbacd7a855f75bae0,wizardlm-13b,[{'content': 'Detailed description of the flui...,2,English,"[{'categories': {'harassment': False, 'harassm...",False
8,a88eb68e2798441f8703dccbee0dea31,vicuna-33b,[{'content': 'There is a new mandate that stat...,6,English,"[{'categories': {'harassment': False, 'harassm...",True
9,c02a471852bf433fab1d241d8db05e53,wizardlm-13b,[{'content': 'ten steps (bulleted format) desc...,2,English,"[{'categories': {'harassment': False, 'harassm...",False


### Querying function

In [27]:
def search_in_files(filter_str, files_list, min_results=1):
    """
    Searches through the given list of parquet files until at least min_results are found.
    
    Parameters:
    - filter_str: SQL condition (without the WHERE keyword) to filter rows.
                        For example: "contains(lower(cast(conversation as VARCHAR)), 'mounting an')"
    - min_results: Minimum number of results to be satisfied before stopping the search.
    - files_list: List of parquet file names to search into.
    
    Returns:
    - A pandas DataFrame with the query results (possibly empty if no file meets the criteria).
    """
    files = files_list.copy()
    random.shuffle(files)
    
    result_df = pd.DataFrame()
    
    for file_name in files:
        url = f'https://huggingface.co/datasets/lmsys/lmsys-chat-1m/resolve/main/data/{file_name}'
        print(f"Querying file: {file_name}")
        r = requests.get(url, headers=headers)
        with tempfile.NamedTemporaryFile(suffix=".parquet", delete=False) as tmp:
            tmp.write(r.content)
            tmp_path = tmp.name
        
        query_str = f"""
            SELECT * FROM read_parquet('{tmp_path}') 
            WHERE contains(lower(cast(conversation as VARCHAR)), '{filter_str}')
            """
        df = duckdb.query(query_str).df()
        print(f"Found {len(df)} result(s) in {file_name}")
        
        if len(df) > 0:
            result_df = pd.concat([result_df, df], ignore_index=True)
            
        if len(result_df) >= min_results:
            break
    
    return result_df

random.shuffle(data_files)
filter_str = "b00bz"
df = search_in_files(filter_str=filter_str, files_list=data_files, min_results=6)
display(df)

Querying file: train-00001-of-00006-4030672591c2f478.parquet
Found 0 result(s) in train-00001-of-00006-4030672591c2f478.parquet
Querying file: train-00002-of-00006-1779b7cec9462180.parquet
Found 0 result(s) in train-00002-of-00006-1779b7cec9462180.parquet
Querying file: train-00005-of-00006-fe1acc5d10a9f0e2.parquet
Found 0 result(s) in train-00005-of-00006-fe1acc5d10a9f0e2.parquet
Querying file: train-00003-of-00006-2fa862bfed56af1f.parquet
Found 5 result(s) in train-00003-of-00006-2fa862bfed56af1f.parquet
Querying file: train-00004-of-00006-18f4bdd50c103e71.parquet
Found 1 result(s) in train-00004-of-00006-18f4bdd50c103e71.parquet


,conversation_id,model,conversation,turn,language,openai_moderation,redacted
0,c71d21e138a549e3bc510dd9ce28abd3,koala-13b,[{'content': 'turn this leet speak into normal...,1,English,"[{'categories': {'harassment': False, 'harassm...",False
1,e437eb3080954eae9f494057722c018d,fastchat-t5-3b,[{'content': 'turn this leet speak into normal...,7,English,"[{'categories': {'harassment': False, 'harassm...",True
2,c444480bb73d47f58e7026b7f8c95028,dolly-v2-12b,[{'content': 'turn this leet speak into normal...,2,English,"[{'categories': {'harassment': False, 'harassm...",False
3,5f2bd20b2cde438b8d3b32e4283928ae,chatglm-6b,[{'content': 'turn this leet speak into normal...,1,English,"[{'categories': {'harassment': False, 'harassm...",False
4,70069683196b47ba9cdaac0af63be2b7,vicuna-13b,[{'content': 'turn this leet speak into normal...,1,English,"[{'categories': {'harassment': False, 'harassm...",False
5,b1ee14c850e54fad9e7c5b3901198255,dolly-v2-12b,[{'content': 'translate this leet speak senten...,3,English,"[{'categories': {'harassment': False, 'harassm...",False


In [29]:
random.shuffle(data_files)
filter_str = "leet speak"
search_in_files(filter_str=filter_str, files_list=data_files, min_results=12)

Querying file: train-00005-of-00006-fe1acc5d10a9f0e2.parquet
Found 1 result(s) in train-00005-of-00006-fe1acc5d10a9f0e2.parquet
Querying file: train-00003-of-00006-2fa862bfed56af1f.parquet
Found 10 result(s) in train-00003-of-00006-2fa862bfed56af1f.parquet
Querying file: train-00004-of-00006-18f4bdd50c103e71.parquet
Found 2 result(s) in train-00004-of-00006-18f4bdd50c103e71.parquet


,conversation_id,model,conversation,turn,language,openai_moderation,redacted
0,87d47f45d88e433bb3f31242bba0d56b,dolly-v2-12b,[{'content': 'turn this leet speak into normal...,3,English,"[{'categories': {'harassment': False, 'harassm...",True
1,049a6200118d45b189be62b79c04d35f,vicuna-13b,[{'content': 'turn this leet speak into normal...,1,English,"[{'categories': {'harassment': False, 'harassm...",False
2,737cf20378f24c5284202a282eedca23,vicuna-13b,[{'content': 'turn this leet speak into normal...,1,English,"[{'categories': {'harassment': False, 'harassm...",False
3,19854eda4814485aae2646974d5d50bd,dolly-v2-12b,[{'content': 'turn this leet speak into normal...,1,English,"[{'categories': {'harassment': False, 'harassm...",False
4,c99b7c72259849ba8cd21038322bfa98,koala-13b,[{'content': 'turn this leet speak into Englis...,1,English,"[{'categories': {'harassment': False, 'harassm...",False
5,c71d21e138a549e3bc510dd9ce28abd3,koala-13b,[{'content': 'turn this leet speak into normal...,1,English,"[{'categories': {'harassment': False, 'harassm...",False
6,f17a2a95d15e461d8dde2f842588db3f,oasst-pythia-12b,"[{'content': 'From now on, you should respond ...",1,English,"[{'categories': {'harassment': False, 'harassm...",False
7,e437eb3080954eae9f494057722c018d,fastchat-t5-3b,[{'content': 'turn this leet speak into normal...,7,English,"[{'categories': {'harassment': False, 'harassm...",True
8,c444480bb73d47f58e7026b7f8c95028,dolly-v2-12b,[{'content': 'turn this leet speak into normal...,2,English,"[{'categories': {'harassment': False, 'harassm...",False
9,5f2bd20b2cde438b8d3b32e4283928ae,chatglm-6b,[{'content': 'turn this leet speak into normal...,1,English,"[{'categories': {'harassment': False, 'harassm...",False
